# 딕셔너리와 집합
dict 형은 애플리케이션에서 널리 사용될 뿐만 아니라 파이썬 구현의 핵심 부분이기도 하다. 모듈 네임스페이스, 클래스 및 인스턴스 속성, 함수의 키워드 인수 등 핵심 부분에 딕셔너리가 사용되고 있다. 내장 합수들은 `__builtins__.__dict__`에 들어 있다.

중요한 역할을 맡고 있으므로 파이썬 dict 클래스는 상당히 최적화되어 있다. 파이썬의 고성능 딕셔너리 뒤에는 해시 테이블이라는 엔진이 있다.

집합도 해시 테이블을 이용해서 구현하므로, 이 장에서는 집합도 다룬다. 해시 테이블이 작동하는 방식을 알아야 딕셔너리와 집합을 최대로 활용할 수 있다.

- 공통적으로 사용되는 딕셔너리 메서드
- 없는 키에 대한 특별 처리
- 표준 라이브러리에서 제공하는 다양한 딕셔너리 클래스
- set과 frozenset 형
- 해시 테이블의 작동 방식
- 해시 테이블의 의미(키 자료형 제한, 예측할 수 없는 순서 등)

## 3.1 일반적인 매핑형
`collections.abc` 모듈은 dict 및 이와 유사한 자료형의 인터페이스를 정의하기 위해 Mapping 및 MutableMapping 추상 베이스 클래스(ABC)를 제공한다.

![dict_mapping](./3-1.png)

특화된 매핑은 여기에 나온 추상 베이스 클래스 대신 dict나 collections.UserDict 클래스를 상속하기도 한다. 추상 베이스 클래스는 매핑이 제공해야 하는 최소한의 인터페이스를 정의하고 문서화하기 위한 것이며, 넓은 의미의 매핑을 지원해야 하는 코드에서 `isinstance()` 테스트를 하기 위한 기준으로 사용된다.

표준 라이브러리에서 제공하는 매핑형은 모두 dict를 이용해서 구현하므로, 키가 ***해시 가능***해야 한다는 제한을 갖고 있다.
>***해시 가능하다(hashable)는 말의 의미는?***
>
수명 주기 동안 결코 변하지 않는 해시값을 갖고 있고(`__hash__()` 메서드가 필요) 다른 객체와 비교할 수 있으면(`__eq__()` 메서드가 필요), 객체를 해시 가능하다고 한다. 동일하다고 판단되는 객체는 반드시 해시값이 동일해야 한다.

In [1]:
a = dict(one=1, two=2, three=3)
b = {'one': 1, 'two': 2, 'three':3}
c = dict(zip(['one', 'two', 'three'], [1, 2, 3]))
d = dict([('two', 2), ('one', 1), ('three', 3)])
e = dict({'three': 3, 'one': 1, 'two': 2})
a == b == c == d == e

True

## 3.2 지능형 딕셔너리

## 3.3 공통적인 매핑 메서드
다음 표는 dict와 dict의 변형 중 가장 널리 사용되는 defaultdict와 OrderedDict 클래스가 구현하는 메서드를 보여준다.
![dict-api](./3-2.png)

### 3.3.1 존재하지 않는 키를 setdefault()로 처리하기
fail-fast 철학에 따라, 존재하지 않는 키 k로 d[k]를 접근하면 dict는 오류를 발생시킨다. KeyError를 처리하는 것보다 기본값을 사용하는 것이 더 편리한 경우에는 d.get(k, default)를 사용한다는 것은 파이썬 개발자라면 누구나 알고 있을 것이다. 그렇지만 발견한 값을 갱신할 때, 해당 객체가 가변 객체면 `__getitem__()`이나 `get()` 메서드는 보기 어색하며, 효율성이 좋지 않다. 다음은 `dict.get()`이 좋지 않은 사례를 보여준다.

In [4]:
import sys
import re

WORD_RE = re.compile('\w+')

index = {}
with open("./zen.txt", encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start()+1
            location = (line_no, column_no)
            # this is ugly; coded like this to make a point
            occurrences = index.get(word, [])  # <1>
            occurrences.append(location)       # <2>
            index[word] = occurrences          # <3>

# print in alphabetical order
for word in sorted(index, key=str.upper):  # <4>
    print(word, index[word])

a [(19, 48), (20, 53)]
Although [(11, 1), (16, 1), (18, 1)]
ambiguity [(14, 16)]
and [(15, 23)]
are [(21, 12)]
aren [(10, 15)]
at [(16, 38)]
bad [(19, 50)]
be [(15, 14), (16, 27), (20, 50)]
beats [(11, 23)]
Beautiful [(3, 1)]
better [(3, 14), (4, 13), (5, 11), (6, 12), (7, 9), (8, 11), (17, 8), (18, 25)]
break [(10, 40)]
by [(1, 20)]
cases [(10, 9)]
Complex [(6, 1)]
complex [(5, 23)]
complicated [(6, 24)]
counts [(9, 13)]
dense [(8, 23)]
do [(15, 64), (21, 48)]
Dutch [(16, 61)]
easy [(20, 26)]
enough [(10, 30)]
Errors [(12, 1)]
explain [(19, 34), (20, 34)]
Explicit [(4, 1)]
explicitly [(13, 8)]
face [(14, 8)]
first [(16, 41)]
Flat [(7, 1)]
good [(20, 55)]
great [(21, 28)]
guess [(14, 52)]
hard [(19, 26)]
honking [(21, 20)]
idea [(19, 54), (20, 60), (21, 34)]
If [(19, 1), (20, 1)]
implementation [(19, 8), (20, 8)]
implicit [(4, 25)]
In [(14, 1)]
is [(3, 11), (4, 10), (5, 8), (6, 9), (7, 6), (8, 8), (17, 5), (18, 16), (19, 23), (20, 23)]
it [(15, 67), (19, 43), (20, 43)]
let [(21, 42)]
m

In [5]:
import sys
import re

WORD_RE = re.compile('\w+')

index = {}
with open("./zen.txt", encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start()+1
            location = (line_no, column_no)
            # this is ugly; coded like this to make a point
            index.setdefault(word, []).append(location)

# print in alphabetical order
for word in sorted(index, key=str.upper):  # <4>
    print(word, index[word])

a [(19, 48), (20, 53)]
Although [(11, 1), (16, 1), (18, 1)]
ambiguity [(14, 16)]
and [(15, 23)]
are [(21, 12)]
aren [(10, 15)]
at [(16, 38)]
bad [(19, 50)]
be [(15, 14), (16, 27), (20, 50)]
beats [(11, 23)]
Beautiful [(3, 1)]
better [(3, 14), (4, 13), (5, 11), (6, 12), (7, 9), (8, 11), (17, 8), (18, 25)]
break [(10, 40)]
by [(1, 20)]
cases [(10, 9)]
Complex [(6, 1)]
complex [(5, 23)]
complicated [(6, 24)]
counts [(9, 13)]
dense [(8, 23)]
do [(15, 64), (21, 48)]
Dutch [(16, 61)]
easy [(20, 26)]
enough [(10, 30)]
Errors [(12, 1)]
explain [(19, 34), (20, 34)]
Explicit [(4, 1)]
explicitly [(13, 8)]
face [(14, 8)]
first [(16, 41)]
Flat [(7, 1)]
good [(20, 55)]
great [(21, 28)]
guess [(14, 52)]
hard [(19, 26)]
honking [(21, 20)]
idea [(19, 54), (20, 60), (21, 34)]
If [(19, 1), (20, 1)]
implementation [(19, 8), (20, 8)]
implicit [(4, 25)]
In [(14, 1)]
is [(3, 11), (4, 10), (5, 8), (6, 9), (7, 6), (8, 8), (17, 5), (18, 16), (19, 23), (20, 23)]
it [(15, 67), (19, 43), (20, 43)]
let [(21, 42)]
m

## 3.4 융통성 있게 키를 조회하는 매핑
검색할 때 키가 존재하지 않으면 어떤 특별한 값을 반환하는 매핑이 있으면 편리한 때가 종종 있다. 이런 딕셔너리를 만드는 방법은 크게 두 가지다. 하나는 평범한 dict 대신 defaultdict를 사용하는 방법이고, 다른 하나는 dict 등의 매핑형을 상속해서 `__missing__()` 메서드를 추가하는 방법이다.

### 3.4.1 defaultdict: 존재하지 않는 키에 대한 또 다른 처리
작동하는 방식은 defaultdict 객체를 생성할 때 존재하지 않는 키 인수로 `__getitem__()` 메서드를 호출할 때마다 기본값을 생성하기 위해 사용되는 콜러블을 제공하는 것이다. 

예를 들어 `dd=defaultdict(list)` 코드로 기본 defaultdict 객체를 생성한 후 ,  `dd`에 존재하지 않는 키인 'new-key'로 `dd['new-key']`표현식을 실행하면 다음과 같이 처리된다.
1. 리스트를 새로 생성하기 위해 list()를 호출한다.
2. 'new-key'를 키로 사용해서 새로운 리스트를 dd에 삽입한다.
3. 리스트에 대한 참조를 반환한다.

### 3.4.2 `__missing__()` 메서드
매핑형은 이름으로도 쉽게 추측할 수 있는 `__missing__()` 메서드를 이용해서 존재하지 않는 키를 처리한다.

In [11]:
class StrKeyDict0(dict):  # <1>

    def __missing__(self, key):
        if isinstance(key, str):  # <2>
            raise KeyError(key)
        return self[str(key)]  # <3>

    def get(self, key, default=None):
        try:
            return self[key]  # <4>
        except KeyError:
            return default  # <5>

    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys()  # <6>

In [12]:
# Tests for item retrieval using `d[key]` notation::

d = StrKeyDict0([('2', 'two'), ('4', 'four')])
d['2']

'two'

In [13]:
d[4]

'four'

In [14]:
d[1]

KeyError: '1'

In [15]:
# Tests for item retrieval using `d.get(key)` notation::

d.get('2')

'two'

In [ ]:
d.get(1, 'N/A')

In [16]:
# Tests for the `in` operator::
print(2 in d)
print(1 in d)

True
False


## 3.5 그 외 매핑형
- collections.OrderedDict
- collections.ChainMap
- collections.Counter
- collections.UserDict

## 3.6  UserDict 상속하기
dict보다는 UserDict를 상속해서 매핑형을 만드는 것이 쉽다. 매핑에 추가한 키를 문자열로 저장하기 위해 UserDict 값을 평가할 수 있다.

내장형에서는 아무런 문제없이 상속할 수 있는 메서드들을 오버라이드해야 하는 구현의 특이성 때문에 dict보다는 UserDict를 상속하는 것이 낫다.

## 3.8 집합이론
파이썬에서 집합은 비교적 최근에 추가되었으며 그리 많이 사용되지는 않는다. set형과 set의 불변형 버전인 frozenset은 파이썬 2.3에 모듈로 처음 등장했으며, 파이썬 2.6에서 내장형으로 승격되었다.

### 3.8.3 집합 연산
수학에서 교집합, 합집합, 차집합 등의 집합 연산은 파이썬에서도 가능하다.

## 3.9 dict와 set의 내부 구조
파이썬이 해시 테이블을 이요해서 딕셔너리와 집합을 구현하는 방식을 알면, 그들의 장점과 단점을 이해하는 데 도움이 된다.
- 파이썬 dict와 set은 얼마나 효율적인가?
- 왜 순서가 없을까?
- dict의 키와 set항목에 파이썬의 모든 객체를 사용할 수 없는 이유는 무엇인가?
- dict의 키와 set 항목의 순서가 왜 삽입 순서에 따라 달라지며, 객체 수명주기 동안 이 순서가 바뀔 수 있는 이유는 무었일까?
- 딕셔너리와 집합을 반복하는 동안 항목을 추가하면 왜 안될까?

### 3.9.1 성능 실험
dict과 set은 빠르다. 왜 빠를까? 해시 테이블의 내부를 이해하면 키들이 무작위 순서로 불안정하게 정렬 되는 이유도 알 수 있다.

### 3.9.2 딕셔너리 안의 해시 테이블
#### 해시와 동치성
두 객체가 동일하면 이 값들의 해시값도 동일해야 한다. 그렇지 않으면 해시 케이블 알고리즘이 제대로 작동하지 않는다. 예를 들어 정수 1과 실수 1의 내부 표현 형태는 다르지만, 1 == 1.0이 참이므로 hash(1) == hash(1.0)도 참이 되어야 한다.

그리고 해시 테이블 인덱스처럼 효율성을 높이려면 해시값이 가능한 한 인덱스 공간에 골고루 퍼져야 한다. 즉, 이상적으로는 비슷하지만 동일하지 않은 객체들의 해시값은 상당히 달라야 한다.

#### 해시 테이블 알고리즘
`my_dict[search_key]`에서 값을 가져오기 위해 파이썬은 `__hash__(search_key)`를 호출해서 search_key의 해시값을 가져오고, 해시값의 최하위 비트를 해시 테이블 안의 버킷에 대한 오프셋으로 사용한다. 찾아낸 버킷이 비어 있으면 KeyError를 발생시키고, 그렇지 않으면 버킷에 들어 있는 항목인 (found_key: found_value) 쌍을 검사해서 search_key == found_key인지 검사한다. 이 값이 일치하면 항목을 찾은 것이므로 found_value를 반환한다.
![hash_crush](./3-3.png)

### 3.9.3 dict 작동 방식에 의한 영향
#### 키 객체는 반드시 해시 가능해야 한다
다음 요구사항을 모두 만족하는 객체는 해시 가능하다.
- 객체의 수명 주기 동안 언제나 동일한 값을 반환하는 __hash__() 메서드를 제공해서 hash() 함수를 지원한다.
- __eq__() 메서드를 통해 동치성을 판단할 수 있다.
- a == b가 참이면, hash(a) == hash(b)도 반드시 참이어야 한다.
#### dict의 메모리 오버헤드가 크다
dict가 내부적으로 해시 테이블을 사용하고 있고 해시가 제대로 작동하려면 빈 공간이 충분해야 하므로, dict의 메모리 공간 효율성은 높지 않다. 수백만 개의 객체를 다루고 있고 컴퓨터의 램이 수 기가바이트라면, 실제로 문제가 발생하기 전까지 이런 최적화 기법을 사용하지 않고 버틸 수 있다.
#### 키 검색이 아주 빠르다
#### 키 순서는 삽입 순서에 따라 달라진다
#### 딕셔너리에 항목을 추가하면 기존 키의 순서가 변경될 수 있다

### 3.9.4 집합의 작동방식 - 현실적으로 미치는 영향
set과 frozenset도 해시 테이블을 이용해서 구현하지만, 각 버킷이 항목에 대한 참조만을 담고 있다는 점이 다르다(항목 자체가 dict에서의 키처럼 사용되지만, 이 키를 통해 접근할 값이 없다).
- set 요소는 모두 해시 가능한 객체여야 한다.
- set의 메모리 오버헤드가 상당히 크다.
- 집합에 속해 있는지 매우 효율적으로 검사할 수 있다.
- 요소의 순서는 요소를 추가한 순서에 따라 달라진다.
- 요소를 집합에 추가하면 다른 요소의 순서가 바뀔 수 있다.

In [ ]:
a = [1,2,2,3,4,4,5]